In [15]:
import importlib
import  cifar_dataloader
importlib.reload(cifar_dataloader)
import  cifar_dataloader
from cifar_dataloader import dataloader2

import train_evaluate
importlib.reload(train_evaluate)
import train_evaluate
from train_evaluate import train,validate,evaluate


import torchvision.models as models
import numpy  as np
import numpy as np
import seaborn as sns
import torch
import torchvision.models as models
import torch.nn as nn




sns.set()

In [16]:
CIFAR = np.load('./datasets/CIFAR.npz')
train_loader,val_loader,test_loader=dataloader2(CIFAR)

(400, 32, 32, 3)


In [12]:
weights = models.ViT_B_16_Weights.DEFAULT
vit = models.vit_b_16(weights=weights)

In [13]:
class VIT(nn.Module):
    def __init__(self, output_shape):
        super(VIT, self).__init__()

        self.pretrained = models.vit_b_16(weights=weights)
        for param in self.pretrained.parameters():
            param.requires_grad = False

        pretrained_output = 1000
        self.fc = torch.nn.Sequential(
            nn.Linear(in_features=pretrained_output,
                      out_features=output_shape,
                      bias=False),
        )

    def forward(self, image):
        output = self.pretrained(image)
        output = self.fc(output)
        return output

vit = VIT(output_shape=3)

In [14]:
def run(dataset,epochs=10,lr=0.01):

    train_loader,val_loader,test_loader=dataloader2(dataset)
    #model=resnet18
    #model=Lenet()
    model=vit
    # resnet18 = resnet18.fc.in_features
    # resnet18.fc = nn.Linear(num_ftrs, 3)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(params=model.parameters(), lr=lr)
    training_losses = []
    training_accuracies = []
    validation_losses = []
    validation_accuracies = []
    test_losses = []
    test_accuracies = []
    
    for epoch in range(epochs):
        train_acc, train_loss=train(model,criterion, optimizer, train_loader)
        val_acc, val_loss=validate(model,criterion, val_loader)
        test_acc, test_loss=validate(model,criterion, test_loader)

        training_accuracies.append(train_acc)
        training_losses.append(train_loss)
        validation_accuracies.append(val_acc)
        validation_losses.append(val_loss)
        test_accuracies.append(test_acc)
        test_losses.append(test_loss)


        print(f"epoch: {epoch}\n \
            train_acc: {train_acc}, train_loss: {train_loss}\n \
            val_acc: {val_acc}, val_loss: {val_loss}\n \
            test_acc: {test_acc}, test_loss: {test_loss}\n")
    precision,recall,accuracy=evaluate(dataset,model)


    return (training_accuracies,training_losses,
            validation_accuracies,
            validation_losses,
            test_accuracies,
            test_losses,
            precision,
            recall,
            accuracy,
            model)

In [ ]:
(training_accuracies,training_losses,
validation_accuracies,
validation_losses,
test_accuracies,
test_losses,
precision,
recall,
accuracy,
model)=run(CIFAR,epochs=1,lr=0.01)